# Recommendation systems

## Collaborative Filtering with Gradient Descent 

I will build a collaborative filtering model to predict netflix ratings.

In [1]:
import numpy as np
import pandas as pd

## Encoding rating data
Here are our very small subset of fake data to get us started.

In [3]:
# The first row says that user 11 reated movie 1 with a score of 4
!cat tiny_training2.csv 

userId,movieId,rating
11,1,4
11,23,5
2,23,5
2,4,3
31,1,4
31,23,4
4,1,5
4,3,2
52,1,1
52,3,4
61,3,5
7,23,1
7,3,3


In [4]:
# here is a handy function from fast.ai
def proc_col(col):
    """Encodes a pandas column with continous ids. 
    """
    uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx[x] for x in col]), len(uniq)

In [5]:
def encode_data(df):
    """Encodes rating data with continous user and movie ids using 
    the helpful fast.ai function from above.
    
    Arguments:
      train_csv: a csv file with columns user_id,movie_id,rating 
    
    Returns:
      df: a dataframe with the encode data
      num_users
      num_movies
      
    """
    encoded_users = proc_col(df['userId'])
    encoded_movies = proc_col(df['movieId'])
    df['userId'] = encoded_users[1]
    df['movieId'] = encoded_movies[1]
    num_users = encoded_users[2]
    num_movies = encoded_movies[2]
    return df, num_users, num_movies

In [6]:
df = pd.read_csv("tiny_training2.csv")
df, num_users, num_movies = encode_data(df)

In [7]:
np.testing.assert_equal(df["userId"].values, np.array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 6, 6]))

## Initializing parameters

In [8]:
def create_embedings(n, K):
    """ Create a numpy random matrix of shape n, K
    
    The random matrix should be initialized with uniform values in (0, 6/K)
    Arguments:
    
    Inputs:
    n: number of items/users
    K: number of factors in the embeding 
    
    Returns:
    emb: numpy array of shape (n, num_factors)
    """
    np.random.seed(3)
    emb = 6*np.random.random((n, K)) / K
    return emb

# here is an example on how the prediction matrix would look like with 7 users and 5 movies
np.dot(create_embedings(7,3), create_embedings(5,3).transpose())

array([[3.55790894, 4.69774849, 0.92361109, 1.58739544, 3.00593239],
       [4.69774849, 7.44656163, 1.18135616, 2.64524868, 4.74559066],
       [0.92361109, 1.18135616, 0.24548062, 0.34025121, 0.69616965],
       [1.58739544, 2.64524868, 0.34025121, 1.61561   , 2.41361975],
       [3.00593239, 4.74559066, 0.69616965, 2.41361975, 3.82505541],
       [2.02000808, 3.29656257, 0.43174569, 2.065911  , 3.07264619],
       [2.07691001, 3.02887291, 0.53270924, 1.02482544, 1.90251125]])

## Encoding Y as a sparse matrix
This code helps you encode a $Y$ as a sparse matrix from the dataframe. 

In [9]:
from scipy import sparse
def df2matrix(df, nrows, ncols, column_name="rating"):
    """ Returns a sparse matrix constructed from a dataframe
    
    This code assumes the df has columns: MovieID,UserID,Rating
    """
    values = df[column_name].values
    ind_movie = df['movieId'].values
    ind_user = df['userId'].values
    return sparse.csc_matrix((values,(ind_user, ind_movie)),shape=(nrows, ncols))

In [10]:
df = pd.read_csv("tiny_training2.csv")
df, num_users, num_movies = encode_data(df)
Y = df2matrix(df, num_users, num_movies)

In [11]:
print(Y)

  (0, 0)	4
  (2, 0)	4
  (3, 0)	5
  (4, 0)	1
  (0, 1)	5
  (1, 1)	5
  (2, 1)	4
  (6, 1)	1
  (1, 2)	3
  (3, 3)	2
  (4, 3)	4
  (5, 3)	5
  (6, 3)	3


## Predicting Ratings

In [12]:
def predict(df, emb_user, emb_movie):
    """ This function computes df["prediction"] without doing (U*V^T).
    
    Compute df["prediction"] by using elementwise multiplication of the corresponding embeddings and then 
    sum to get the prediction u_i*v_j. This avoids creating the dense matrix U*V^T.
    """
    df['prediction'] = np.sum(np.multiply(emb_user[df['userId']],
                                           emb_movie[df['movieId']]), axis=1)
    return df

In [13]:
emb_user = np.ones((num_users, 3))
emb_movie = np.ones((num_movies, 3))
df = predict(df, emb_user, emb_movie)

In [14]:
emb_user = create_embedings(num_users, K=4)
emb_movie = create_embedings(num_movies, K=4)
df = predict(df, emb_user, emb_movie)
pred_12 = df["prediction"][12]

## Calculating the cost function

In [15]:
def cost(df, emb_user, emb_movie):
    """ Computes mean square error
    
    First compute prediction using the predict function.
    Prediction for user i and movie j is emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      error(float): this is the MSE
    """
    df_pred = predict(df, emb_user, emb_movie)
    rating = df_pred['rating']
    pred = df_pred['prediction']
    error = np.mean(np.square(np.add(rating, -pred)))
    return error

In [16]:
emb_user = np.ones((num_users, 3))
emb_movie = np.ones((num_movies, 3))
error = cost(df, emb_user, emb_movie)

In [17]:
emb_user = create_embedings(num_users, K=4)
emb_movie = create_embedings(num_movies, K=4)
error = cost(df, emb_user, emb_movie)

## Calculating gradient

In [18]:
def finite_difference(df, emb_user, emb_movie, ind_u=None, ind_m=None, k=None):
    """ Computes finite difference on MSE(U, V).
    
    This function is used for testing the gradient function. 
    """
    e = 0.000000001
    c1 = cost(df, emb_user, emb_movie)
    K = emb_user.shape[1]
    x = np.zeros_like(emb_user)
    y = np.zeros_like(emb_movie)
    if ind_u is not None:
        x[ind_u][k] = e
    else:
        y[ind_m][k] = e
    c2 = cost(df, emb_user + x, emb_movie + y)
    return (c2 - c1)/e

In [19]:
def gradient(df, emb_user, emb_movie):
    """ Computes the gradient.
    
    Hint: First compute df["prediction"]. Then use df2matrix
    to get a sparse matrix Y and Y_hat.
    
    Arguments:
      df: dataframe with all data or a subset of the data
      Y: sparse representation of df
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      d_emb_user
      d_emb_movie
    """
    df_pred = predict(df, emb_user, emb_movie)
    num_users = emb_user.shape[0]
    num_movies = emb_movie.shape[0]
    Y = df2matrix(df, num_users, num_movies)
    Y_hat = df2matrix(df_pred, num_users, num_movies, column_name="prediction")
    d_emb_user = -2/len(df)*(Y-Y_hat)*emb_movie
    d_emb_movie = -2/len(df)*(Y-Y_hat).T*emb_user
    return d_emb_user, d_emb_movie

In [20]:
K = 3
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
grad_user, grad_movie = gradient(df, emb_user, emb_movie)

In [23]:
user=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_u=user, k=i) for i in range(K)])
assert(np.all(np.abs(grad_user[user] - approx) < 0.0001))

In [24]:
movie=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_m=movie, k=i) for i in range(K)])
assert(np.all(np.abs(grad_movie[movie] - approx) < 0.0001))

## Using gradient descent with momentum

In [25]:
def gradient_descent(df, emb_user, emb_movie, iterations=100, learning_rate=0.01, df_val=None):
    """ Computes gradient descent with momentum (0.9) for a number of iterations.
    
    Prints training cost and validation cost (if df_val is not None) every 50 iterations.
    
    Returns:
    emb_user: the trained user embedding
    emb_movie: the trained movie embedding
    """
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    beta = 0.9
    d_emb_user, d_emb_movie = gradient(df, emb_user, emb_movie)
    for i in range(iterations):
        df = predict(df, emb_user, emb_movie)
        Y_hat = df2matrix(df, emb_user.shape[0], emb_movie.shape[0], column_name="prediction")
        grad_user, grad_movie = gradient(df, emb_user, emb_movie)
        d_emb_user = beta*d_emb_user + (1-beta)*grad_user
        d_emb_movie = beta*d_emb_movie + (1-beta)*grad_movie
        emb_user -= learning_rate*d_emb_user
        emb_movie -= learning_rate*d_emb_movie
        if (i+1)%50 == 0:
            print(cost(df, emb_user, emb_movie))
            if df_val is not None:
                print(cost(df_val, emb_user, emb_movie))
    return emb_user, emb_movie

In [26]:
emb_user = create_embedings(num_users, 3)
emb_movie = create_embedings(num_movies, 3)
emb_user, emb_movie = gradient_descent(df, emb_user, emb_movie, iterations=200, learning_rate=0.01)

1.701364306188372
0.9748705172056421
0.6991455656925722
0.5265621094145296


In [27]:
train_mse = cost(df, emb_user, emb_movie)

## Predicting on new data
Now we should write a function that given new data is able to predict ratings. First we write a function that encodes new data. If a new user or item is present that row should be remove. Collaborative Filtering is not good at handling new users or new items. To help with this task, you could write a an auxiliary function similar to `proc_col`.

In [28]:
def encode_new_data(df_val, df_train):
    """ Encodes df_val with the same encoding as df_train.
    Returns:
    df_val: dataframe with the same encoding as df_train
    """
    user2idx = proc_col(df_train['userId'])[0]
    movie2idx = proc_col(df_train['movieId'])[0]
    df_val = df_val[df_val['userId'].isin(user2idx.keys())]
    df_val = df_val[df_val['movieId'].isin(movie2idx.keys())]
    df_val['userId'] = [user2idx[x] for x in df_val['userId']]
    df_val['movieId'] = [movie2idx[x] for x in df_val['movieId']]
    return df_val

In [29]:
df_t = pd.read_csv("tiny_training2.csv")
df_v = pd.read_csv("tiny_val2.csv")
df_v = encode_new_data(df_v, df_t)

## Putting it all together
For this part you should get data from here
`wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip`

In [36]:
data = pd.read_csv("ml-latest-small/ratings.csv")
# sorting by timestamp take as validation data the most recent data doesn't work so let's just take 20%
# at random
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8
train = data[msk].copy()
val = data[~msk].copy()
df_train, num_users, num_movies = encode_data(train.copy())
df_val = encode_new_data(val.copy(), train.copy())
print(len(val), len(df_val))

20386 19591


In [37]:
K = 50
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=200, learning_rate=1, df_val=df_val)

9.48094555948595
9.60843885425874
6.6913902077377765
6.824174774852403
4.803840866998358
4.9284672300475245
3.7567143651597688
3.868546473763612


In [38]:
train_mse = cost(df_train, emb_user, emb_movie)
val_mse = cost(df_val, emb_user, emb_movie)
print(train_mse, val_mse)

3.7567143651597688 3.868546473763612
